# Prepare synthetic data for simulating health information in normal and abnormal behavior

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
normal_cpu_utilization = np.random.normal(0.6,0.1, 100)
abnormal_cpu_utilization = np.random.normal(0.9,0.1, 10)
normal_free_heap_percent = np.random.normal(0.4,0.1, 100)
normal_jvm_old_generation_used = np.random.normal(0.5,0.2, 100)
# normal_cpu_utilization

## Features Identified

feature 1: avgbackground queue, (1500) [tasknode] <br>
feature 2: avg workflow queue, (55) [tasknode]<br>
feature 3: avg node manager thread pool size, (5.57) [tasknode] [feature 1,2,3 are related]<br>
 <br>
feature 4 : node id (need to revisit) [can we put node id as feature so that we can backtrack it later ???]<br>
feature 5: index=mail, failed post output=ssl ctx set options [source : splunk search] ( 0 normal, 5000+ abnormal)<br>
feature 6: product release (value 0,1)<br>
feature 7: ui node thread count (normal 250, abnormal 500+) (ims990)<br>
feature 8: ariba.ui.aribaweb.util.AWGenericException: java.lang.IllegalStateException: splunk search linked with feature 7<br>
feature 9: scheduletaskstatustab query arches batch publishinselectedrealm job exception <br>
<br>
feature 10: catalogue search time.(38) linked with 1,2,3 feature<br>
feature 11: no network connectivity between SNV and US1 - ims-983<br>
feature 12: 4310024, "Failed to get JDBC connection permit ims-980 (normal 0-100, abnormal 100-1000)<br>
feature 13: node status fair to critical its-980 (linked with feature 12)<br>
feature 14: cloud health index down more than N time in X minutes<br>
<br>
Feature 15: JVM Heap size getting filled & garbage collector not able to clean the data & frequently garbage collector is running<br>
Feature 16: log size getting filled drastically 10%->100% in an hour<br>
Feature 17: GT node restarting [Need more data]<br>
</p>

RecordDate  - Record date and time <br>

In [4]:
columns = ['RecordDate','AvgBackgroundQ', 'AvgThreadPoolSize', 'AvgWorkflowQ', 'CatalogSearchTime', 
           'Exception', 'LogSizeVolumePercent', 'NetworkConnectivitySNV-US1',  'NodeId', 'IsProductReleased',
          'UiNodeThreadsCount','CloudHealthIndex', 'Label'  
          ]

print('Dataset columns \n' , columns)

Dataset columns 
 ['RecordDate', 'AvgBackgroundQ', 'AvgThreadPoolSize', 'AvgWorkflowQ', 'CatalogSearchTime', 'Exception', 'LogSizeVolumePercent', 'NetworkConnectivitySNV-US1', 'NodeId', 'IsProductReleased', 'UiNodeThreadsCount', 'CloudHealthIndex', 'Label']


In [5]:
recordsCount = 10000

In [6]:
# Average BackgroudQ value is around 100-300 for normal & 1000+ abnormally
avgBackgroundQ = np.round(np.random.normal(200, 100, recordsCount-100)).astype(int)

# add abnormal values
avgBackgroundQ = np.append(avgBackgroundQ, np.round(np.random.normal(1000, 100, 100)).astype(int))


#Average WorkflowQ value is around 10-40
avgWorkflowQ = np.round(np.random.normal(20, 10, recordsCount)).astype(int)

#Average ThreadPoolSize would be around 8-20
avgThreadPoolSize = np.round(np.random.normal(8, 4, recordsCount)).astype(int)

# Most of the records don't have ID -1 represents no nodeid
nodeId = np.random.choice(np.append(np.arange(60,80), [-999]),recordsCount)

# Indicates if product is released
isProductReleased = np.random.choice([0,1], recordsCount)

# UI health index data 
uiNodeThreadCount = np.round(np.random.normal(250, 200, recordsCount)).astype(int)

# CloudHealthIndex
cloudHealthValues = ['FAIR', 'GOOD', 'CRITICAL']
cloudHealthIndex = np.random.choice(cloudHealthValues, recordsCount)

# Exception occurred
exception = np.random.choice(['AWGenericException: java.lang.IllegalStateException','java.lang.OutOfMemoryException',
                              'NA', 'javax.net.ssl.SSLHandshakeException', 'spanning tree event',
                              'JDBC-connection-permit-failure', 'OutOfMemoryException, GT Nodes restarting]'], recordsCount)

# Catelog search time
catalogSearchTime = np.round(np.random.normal(20, 10, recordsCount)).astype(int)

# Network Connectivity broken ?
nwConnectivitySNV_US1 = np.random.choice([0,1], recordsCount)

# LogSize Volume Percent 0-1 . 
logSizeVolumePercent = np.round(np.random.random(recordsCount),decimals=3)
#np.random.randint(50,100,recordsCount)

# Record dates for every 30min from 1 Jan 2017, taking only given number of records
record_dates = pd.date_range(pd.to_datetime('01-01-2017', format='%d-%m-%Y'), 
                                  pd.to_datetime('31-12-2017', format='%d-%m-%Y'), freq='30min')[:recordsCount]


print('avgBackgroundQ = ', avgBackgroundQ)
print('avgWorkflowQ = ', avgWorkflowQ)
print('avgThreadPoolSize = ', avgThreadPoolSize)
print('nodeId = ', nodeId)
print('isProductReleased = ', isProductReleased)
print('uiNodeThreadCount = ', uiNodeThreadCount)
print('cloudHealthIndex = ', cloudHealthIndex)
print('exception = ', exception)
print('catalogSearchTime = ', catalogSearchTime)
print('nwConnectivitySNV_US1 = ', nwConnectivitySNV_US1)
print('logSizeVolumePercent = ', logSizeVolumePercent)
print('record_dates =\n', record_dates)


avgBackgroundQ =  [ 210  347  179 ... 1023  943 1046]
avgWorkflowQ =  [ 1 22 31 ... 14 41 24]
avgThreadPoolSize =  [ 4  9 -2 ...  4 11  6]
nodeId =  [61 71 79 ... 68 66 73]
isProductReleased =  [0 1 0 ... 0 0 1]
uiNodeThreadCount =  [318 327 493 ...  60 403 249]
cloudHealthIndex =  ['CRITICAL' 'CRITICAL' 'FAIR' ... 'FAIR' 'GOOD' 'CRITICAL']
exception =  ['JDBC-connection-permit-failure' 'javax.net.ssl.SSLHandshakeException'
 'JDBC-connection-permit-failure' ... 'JDBC-connection-permit-failure'
 'NA' 'java.lang.OutOfMemoryException']
catalogSearchTime =  [18 44 20 ... 19 19 27]
nwConnectivitySNV_US1 =  [0 1 0 ... 0 1 1]
logSizeVolumePercent =  [0.337 0.487 0.828 ... 0.166 0.981 0.485]
record_dates =
 DatetimeIndex(['2017-01-01 00:00:00', '2017-01-01 00:30:00',
               '2017-01-01 01:00:00', '2017-01-01 01:30:00',
               '2017-01-01 02:00:00', '2017-01-01 02:30:00',
               '2017-01-01 03:00:00', '2017-01-01 03:30:00',
               '2017-01-01 04:00:00', '2017-01-

In [7]:
# There are fixed Issue category

incidentTypes = ['NoIssue', 'DBConnectionIssue', 'InvoiceIssue', 'OrderIssue', 'CommunityIssue',
                 'WorkspaceIssue' , 'NetworkIssue', 'CommunityHealthIssue' ]

labels = np.random.choice(incidentTypes, recordsCount)
print('labels = ', +labels)


labels =  ['NetworkIssue' 'NetworkIssue' 'OrderIssue' ... 'CommunityIssue'
 'CommunityHealthIssue' 'InvoiceIssue']


In [8]:
recordsDict= {
    'AvgBackgroundQ' : avgBackgroundQ, 'AvgWorkflowQ' : avgWorkflowQ, 'AvgThreadPoolSize' : avgThreadPoolSize,
    'NodeId' : nodeId, 'IsProductReleased' : isProductReleased, 'UiNodeThreadsCount':uiNodeThreadCount , 
     'CatalogSearchTime' : catalogSearchTime, 
    'NetworkConnectivitySNV-US1': nwConnectivitySNV_US1, 'LogSizeVolumePercent' : logSizeVolumePercent,
    'Label' : labels, 'Date' : record_dates, 'CloudHealthIndex' : cloudHealthIndex
}
# 'Exception' : exception, is removed for now

sample1 = pd.DataFrame(data=recordsDict)
sample1.head()

,AvgBackgroundQ,AvgWorkflowQ,AvgThreadPoolSize,NodeId,IsProductReleased,UiNodeThreadsCount,CatalogSearchTime,NetworkConnectivitySNV-US1,LogSizeVolumePercent,Label,Date,CloudHealthIndex
0,210,1,4,61,0,318,18,0,0.337,NetworkIssue,2017-01-01 00:00:00,CRITICAL
1,347,22,9,71,1,327,44,1,0.487,NetworkIssue,2017-01-01 00:30:00,CRITICAL
2,179,31,-2,79,0,493,20,0,0.828,OrderIssue,2017-01-01 01:00:00,FAIR
3,241,7,6,71,1,364,35,1,0.641,WorkspaceIssue,2017-01-01 01:30:00,CRITICAL
4,193,5,6,64,1,314,12,0,0.668,InvoiceIssue,2017-01-01 02:00:00,CRITICAL


In [9]:
sample1.to_excel('SystemLogsDataset-temp.xlsx',index=False)

## SystemLogsDataset is created
#### Now we will train the model

### Feature Generation & Normalization

In [10]:
# Consider all negative values for AvgBackgroundQ, AvgThreadPoolSize, AvgWorkflowQ, CatalogSearchTime, UiNodeThreadsCount
# as Missing values and replace with -999

df = sample1.copy()

df['AvgBackgroundQ'][df['AvgBackgroundQ'] < 1] = np.NAN
df['AvgThreadPoolSize'][df['AvgThreadPoolSize'] < 1] = np.NAN
df['AvgWorkflowQ'][df['AvgWorkflowQ'] < 1] = np.NAN
df['CatalogSearchTime'][df['CatalogSearchTime'] < 1] = np.NAN
df['UiNodeThreadsCount'][df['UiNodeThreadsCount'] < 1] = np.NAN

# df['AvgBackgroundQ'].hist()
# df['AvgThreadPoolSize'].hist()
# df['AvgWorkflowQ'].hist()
# df['CatalogSearchTime'].hist()
# df['UiNodeThreadsCount'].hist()

# Fill NaN values with -999
df = df.fillna(-999)


#Assign number to label
df['Label'] = df['Label'].apply(incidentTypes.index)
df['CloudHealthIndex'] = df['CloudHealthIndex'].apply(cloudHealthValues.index)

# Create new features from date
df['Day'] = df['Date'].dt.day
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
df['Hour'] = df['Date'].dt.hour
df['Minute'] = df['Date'].dt.minute
df['Second'] = df['Date'].dt.second

df['DayOfYear'] = df['Date'].dt.dayofyear
df['DayOfWeek'] = df['Date'].dt.dayofweek
df['WeekOfYear'] = df['Date'].dt.weekofyear

df['WeekOfYear'] = df['Date'].dt.weekofyear
df['Quarter'] = df['Date'].dt.quarter

df['IsWeekend'] = (df['Date'].dt.dayofweek > 4).astype(int)
df['IsMonthStart'] = (df['Date'].dt.is_month_start).astype(int)
df['IsMonthEnd'] = (df['Date'].dt.is_month_end).astype(int)

df.head()



c:\softwares\python\python36-32\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\softwares\python\python36-32\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
c:\softwares\python\python36-32\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\softwares\python\python36-32\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is try

,AvgBackgroundQ,AvgWorkflowQ,AvgThreadPoolSize,NodeId,IsProductReleased,UiNodeThreadsCount,CatalogSearchTime,NetworkConnectivitySNV-US1,LogSizeVolumePercent,Label,...,Hour,Minute,Second,DayOfYear,DayOfWeek,WeekOfYear,Quarter,IsWeekend,IsMonthStart,IsMonthEnd
0,210.0,1.0,4.0,61,0,318.0,18.0,0,0.337,6,...,0,0,0,1,6,52,1,1,1,0
1,347.0,22.0,9.0,71,1,327.0,44.0,1,0.487,6,...,0,30,0,1,6,52,1,1,1,0
2,179.0,31.0,-999.0,79,0,493.0,20.0,0,0.828,3,...,1,0,0,1,6,52,1,1,1,0
3,241.0,7.0,6.0,71,1,364.0,35.0,1,0.641,5,...,1,30,0,1,6,52,1,1,1,0
4,193.0,5.0,6.0,64,1,314.0,12.0,0,0.668,2,...,2,0,0,1,6,52,1,1,1,0


In [12]:
features = df.columns.drop(['Label', 'Date'])

X = df[features]
y = df['Label']

X.head()

,AvgBackgroundQ,AvgWorkflowQ,AvgThreadPoolSize,NodeId,IsProductReleased,UiNodeThreadsCount,CatalogSearchTime,NetworkConnectivitySNV-US1,LogSizeVolumePercent,CloudHealthIndex,...,Hour,Minute,Second,DayOfYear,DayOfWeek,WeekOfYear,Quarter,IsWeekend,IsMonthStart,IsMonthEnd
0,210.0,1.0,4.0,61,0,318.0,18.0,0,0.337,2,...,0,0,0,1,6,52,1,1,1,0
1,347.0,22.0,9.0,71,1,327.0,44.0,1,0.487,2,...,0,30,0,1,6,52,1,1,1,0
2,179.0,31.0,-999.0,79,0,493.0,20.0,0,0.828,0,...,1,0,0,1,6,52,1,1,1,0
3,241.0,7.0,6.0,71,1,364.0,35.0,1,0.641,2,...,1,30,0,1,6,52,1,1,1,0
4,193.0,5.0,6.0,64,1,314.0,12.0,0,0.668,2,...,2,0,0,1,6,52,1,1,1,0


### Training a model 

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y)

# K-NN 
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
knn.score(X_test, y_test)


0.1272

## Saving a model

In [12]:
import pickle
import dill as pickle

filename = 'model_v1.pk'

with open('models/'+filename, 'wb') as file:
    pickle.dump(knn, file)

## Loading a model

In [13]:
with open('models/'+filename ,'rb') as f:
    loaded_model = pickle.load(f)
    
loaded_model.score(X_test, y_test)

0.1272

In [14]:
predicitons = loaded_model.predict(X_test)
predNames = pd.Series(predicitons).apply(lambda x : incidentTypes[x])
predNames.head()

0         NetworkIssue
1    DBConnectionIssue
2           OrderIssue
3              NoIssue
4              NoIssue
dtype: object

In [15]:
import json
from flask import jsonify
# json.dumps(df.iloc[:1])
inp = sample1.iloc[:1].to_json(orient='records')
inp

'[{"AvgBackgroundQ":164,"AvgWorkflowQ":18,"AvgThreadPoolSize":10,"NodeId":76,"IsProductReleased":1,"UiNodeThreadsCount":1,"CatalogSearchTime":9,"NetworkConnectivitySNV-US1":1,"LogSizeVolumePercent":0.232,"Label":"NetworkIssue","Date":1483228800000,"CloudHealthIndex":"FAIR"}]'

In [16]:
pd.read_json(inp)

,AvgBackgroundQ,AvgThreadPoolSize,AvgWorkflowQ,CatalogSearchTime,CloudHealthIndex,Date,IsProductReleased,Label,LogSizeVolumePercent,NetworkConnectivitySNV-US1,NodeId,UiNodeThreadsCount
0,164,10,18,9,FAIR,2017-01-01,1,NetworkIssue,0.232,1,76,1


In [17]:
jsn1 = "[{'AvgBackgroundQ': 302, 'AvgThreadPoolSize': 7, 'AvgWorkflowQ': 32, 'CatalogSearchTime': 10, 'CloudHealthIndex': 'CRITICAL', 'Date': 1483228800000, 'IsProductReleased': 0, 'Label': 'NetworkIssue', 'LogSizeVolumePercent': 0.087, 'NetworkConnectivitySNV-US1': 1, 'NodeId': 63, 'UiNodeThreadsCount': 403}]"

In [18]:
jsn1

"[{'AvgBackgroundQ': 302, 'AvgThreadPoolSize': 7, 'AvgWorkflowQ': 32, 'CatalogSearchTime': 10, 'CloudHealthIndex': 'CRITICAL', 'Date': 1483228800000, 'IsProductReleased': 0, 'Label': 'NetworkIssue', 'LogSizeVolumePercent': 0.087, 'NetworkConnectivitySNV-US1': 1, 'NodeId': 63, 'UiNodeThreadsCount': 403}]"

In [19]:
jsn1.replace("'", '"')

'[{"AvgBackgroundQ": 302, "AvgThreadPoolSize": 7, "AvgWorkflowQ": 32, "CatalogSearchTime": 10, "CloudHealthIndex": "CRITICAL", "Date": 1483228800000, "IsProductReleased": 0, "Label": "NetworkIssue", "LogSizeVolumePercent": 0.087, "NetworkConnectivitySNV-US1": 1, "NodeId": 63, "UiNodeThreadsCount": 403}]'

In [20]:
df.empty

False

In [21]:
sample1.columns

Index(['AvgBackgroundQ', 'AvgWorkflowQ', 'AvgThreadPoolSize', 'NodeId',
       'IsProductReleased', 'UiNodeThreadsCount', 'CatalogSearchTime',
       'NetworkConnectivitySNV-US1', 'LogSizeVolumePercent', 'Label', 'Date',
       'CloudHealthIndex'],
      dtype='object')

In [27]:
sr = pd.Series(data=[0,1,0,1,1,0])
dict = {0 : 'False', 1 : 'True'}

In [35]:
sr.replace(to_replace=dict)

dd = pd.DataFrame(data=sr, columns=['code'])
dd['Name'] = dd['code'].replace(to_replace=dict)

dd.to_json(orient='records')

,code,Name
0,0,False
1,1,True
2,0,False
3,1,True
4,1,True
5,0,False


### Working on Venky's Data

In [4]:
vf = pd.read_excel('SampleDataset2606v1.xlsx')
vf.head()

,AvgBackgroundQ,AvgThreadPoolSize,AvgWorkflowQ,CatalogSearchTime,Exception,LogSizeVolumePercent,NetworkConnectivitySNV-US1,NodeId,ProductReleased,UiNodeThreadsCount,CloudHealthIndex,Label
0,300,8,15,12,spanning tree event,35,1,NaN,0,25,FAIR,NetworkIssue
1,200,10,19,11,NaN,32,0,NaN,0,12,GOOD,NoIssue
2,250,11,12,11,NaN,43,0,NaN,1,50,GOOD,NoIssue
3,300,5,13,12,NaN,100,0,NaN,0,21,FAIR,DatabaseConnection
4,300,6,21,21,JDBC-connection-permit-failure,28,0,64.0,0,360,CRITICAL,CommunityHealthIssue
